Imports

In [47]:
from pickle import load
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

Load Data

In [48]:
#Import the Payroll Data
df_test = pd.read_excel('GCC Dump 2.xlsx')

#Import the sector reference table
ref_table = pd.read_excel('GCC_ref.xlsx')

df_test.shape

(158584, 8)

Load Models

In [49]:
#Load Models
model_JF=load(open('GCCJF.pkl', 'rb'))
model_Level=load(open('GCCLevel.pkl', 'rb'))

#Load Supporting Elements
vectorizers_JF=load(open('GCCJF_vect.pkl', 'rb'))
vectorizers_Level=load(open('GCCLevel_vect.pkl', 'rb'))

Static Variables

In [50]:
train_cols = ['Co. Designation', 'Co. Function']

In [51]:
target_col = 'Job Family Code'
target_col_2='Level'
jfcol='Job Family Code'
uid = 'Unique ID'

In [52]:
all_cols = ['Co. Designation', 'Co. Function', 'Job Family Code','Level']
univ_cols= ['Co. Designation', 'Co. Function', 'Job Family Code','Level','TFP']

In [53]:
#df_test=df_test.dropna().reset_index(drop=True)
df_backup=df_test.copy(deep=True)
df_test_2=df_test.copy(deep=True)
df_test_2=df_test[univ_cols]
df_test = df_test[all_cols]

Data Preprocessing JF

In [54]:
X_test_unseen = []

for i, col in enumerate(train_cols):
    vectorizer = vectorizers_JF[i]
    X_test_col = vectorizer.transform(df_test.loc[:, col])
    X_test_unseen.append(X_test_col)

X_unseen = hstack(X_test_unseen)

JF Predictions

In [55]:
y_test_pred = model_JF.predict (X_unseen)

In [56]:
y_test_pred_copy=pd.DataFrame(y_test_pred)
y_test_pred_copy.columns = ['Job Family Code']

Data Preprocessing Level

In [57]:
X_test_unseen_2 = []
z=0
for i, col in enumerate(train_cols):
    vectorizer = vectorizers_Level[i]
    X_test_col_2 = vectorizer.transform(df_test.loc[:, col])
    X_test_unseen_2.append(X_test_col_2)
    z=i

vectorizer = vectorizers_Level[z+1]
X_test_col_2 = vectorizer.transform(y_test_pred_copy.loc[:, jfcol])
X_test_unseen_2.append(X_test_col_2)

X_unseen_2 = hstack(X_test_unseen_2)


Level Predictions

In [58]:
y_test_pred_2 = model_Level.predict (X_unseen_2)

In [59]:
pd.DataFrame(y_test_pred).to_csv('JF_x.csv')
pd.DataFrame(y_test_pred_2).to_csv('Lvl_x.csv')
frames=[pd.DataFrame(y_test_pred),pd.DataFrame(y_test_pred_2)]
maps=pd.concat(frames, axis=1)

Level Correction Preprocessing

In [60]:
temp=df_test_2.loc[:,'TFP']
unique_id=df_backup.loc[:,uid]
init_maps=pd.concat([maps , temp], axis=1)
init_maps=pd.concat([unique_id,init_maps], axis=1)
init_maps

,Unique ID,0,0,TFP
0,TPPS Finalized payrolls dump_Part 1.xlsx7954HL,ASXX,I2,4.000000e+06
1,TPPS Finalized payrolls dump_Part 1.xlsx7953HL,ASXX,I2,2.400000e+06
2,TPPS Finalized payrolls dump_Part 1.xlsx7952HL,ASXX,I2,7.000000e+06
3,TPPS Finalized payrolls dump_Part 1.xlsx7951HL,ASXX,I2,4.500000e+06
4,TPPS Finalized payrolls dump_Part 1.xlsx7950HL,ASXX,I2,7.500000e+05
...,...,...,...,...
158579,TPPS Finalized payrolls dump_Part 2.xlsx12Mova...,ASXX,I4,4.669272e+05
158580,TPPS Finalized payrolls dump_Part 2.xlsx12Mova...,ASXX,I4,8.552998e+05
158581,TPPS Finalized payrolls dump_Part 2.xlsx12Mova...,ASXX,I4,5.453952e+05
158582,TPPS Finalized payrolls dump_Part 2.xlsx12Mova...,ASXX,I4,3.564455e+05


In [61]:
init_maps.columns = ['Unique ID','JF', 'level', 'TFP']

In [62]:
def find_in_ref(df, jf, level):
  try:
    return df.loc[(df.JF == jf) & (df.Level == level)]
  except:
    return None

def check_range_ref(ref_df, comp, index):
  ref_row = ref_df.loc[ref_df.index == index]

  if comp >= ref_row.Min.values[0] and comp <= ref_row.Max.values[0] :
    return True

  return False

def compare_range_ref(ref_df, comp, index):
  ref_row = ref_df.loc[ref_df.index == index]

  if comp < ref_row.Min.values[0]:
    return -1
  elif comp > ref_row.Max.values[0]:
    return 1
  elif comp >= ref_row.Min.values[0] and comp <= ref_row.Max.values[0] :
    return 0

  return False


In [63]:
reassigned_maps = init_maps.copy(deep=True)

Level Correction

In [64]:
from tqdm.auto import tqdm

In [65]:
ref_table.loc[ref_table.index == 0].JF.values[0]

'AAAA'

In [67]:
for index, query in tqdm(init_maps.iterrows()):
    if(df_backup.loc[df_backup.index == index,'Co. Designation'].values[0]=='NA' or df_backup.loc[df_backup.index == index,'Co. Function'].values[0]=='NA'):
        init_maps.at[index, 'JF'] = df_backup.loc[df_backup.index == index,'Job Family Code'].values[0]
        query.JF=df_backup.loc[df_backup.index == index,'Job Family Code'].values[0]

0it [00:00, ?it/s]

In [68]:
for index, query in tqdm(init_maps.iterrows()):
  if(df_backup.loc[df_backup.index == index,'Job Family Code'].values[0][0:2]==query.JF[0:2]):
     reassigned_maps.at[index, 'JF'] = df_backup.loc[df_backup.index == index,'Job Family Code'].values[0]
  ref_row = find_in_ref(ref_table, query.JF, query.level)
  d_index=index
  try:
    ref_index = ref_row.index[0]
  except:
    continue

  # check if comp falls in that range
  comp = query['TFP']/1000
  final_index = -1
  cur_jf=query.JF
  if compare_range_ref(ref_table, comp, ref_index)==0:
    #print("Exact match found at ", ref_index)
    final_index = ref_index
  elif compare_range_ref(ref_table, comp, ref_index )==-1:
    ref_index_l=ref_index
    while ref_table.loc[ref_table.index == ref_index_l].JF.values[0]==cur_jf:
      if ref_table.loc[ref_table.index == ref_index_l].Level.values[0]=='S2':
        final_index = ref_index_l
        break
      elif compare_range_ref(ref_table, comp, ref_index_l)==0:
        final_index = ref_index_l
        break
      ref_index_l=ref_index_l-1
  elif compare_range_ref(ref_table, comp, ref_index )==1:
    ref_index_u=ref_index
    while ref_table.loc[ref_table.index == ref_index_u].JF.values[0]==cur_jf:
      if compare_range_ref(ref_table, comp, ref_index_u)==0:
        final_index = ref_index_u
        break
      elif ref_table.loc[ref_table.index == ref_index_u].Level.values[0]=='E2':
        final_index = ref_index_u
        break
      else:
        if ref_table.loc[ref_table.index == ref_index_u].Level.values[0]=='I7':
          ref_index_u = ref_index_u+7
        else:
          ref_index_u = ref_index_u+1

  if final_index != -1 :
    final_level = ref_table.loc[ref_table.index == final_index].Level.values[0]
    reassigned_maps.at[d_index, 'level'] = final_level


0it [00:00, ?it/s]

Final Outputs

In [69]:
pd.DataFrame(init_maps).to_csv('init_maps.csv')
pd.DataFrame(reassigned_maps).to_csv('reassigned maps.csv')